<a href="https://colab.research.google.com/github/JuanJopz08/AnaliticaNeg/blob/main/CarCrash.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
from sklearn.naive_bayes import GaussianNB

# Cargar la base de datos
file_path = '/content/1. BD2_CarCrash_Int.xlsx'
df_total = pd.read_excel(file_path)

# Filtrar por condado de San Francisco
df_sf = df_total[df_total['County'] == 'SAN FRANCISCO']

# Seleccionar variables de interés
df_sf_cortado = df_sf[['Weekday', 'ViolCat', 'Daylight', 'Month', 'CrashType', 'ClearWeather', 'Highway']]

# Convertir CrashType a valores numéricos según orden alfabético
df_sf_cortado.loc[:, 'CrashType'] = df_sf_cortado['CrashType'].astype('category').cat.codes
print(df_sf_cortado['CrashType'])
# Eliminar registros con datos faltantes
df_sf_cortado.dropna(inplace=True)

# Definir variables de entrada y salida
XD = df_sf_cortado[['Weekday', 'ViolCat', 'Daylight', 'Month', 'CrashType', 'ClearWeather']]
yd = df_sf_cortado[['Highway']]

# Entrenar modelo Naive Bayes
mnb = GaussianNB()
mnb.fit(XD, yd.values.ravel())

# Obtener información del modelo
ct = mnb.class_count_
print("Distribución de los datos:", ct)
print("Total de datos:", len(XD))
print("Porcentaje por categoría:", ct / len(XD))

# Calcular medias y desviaciones estándar por categoría
u = mnb.theta_
std = np.sqrt(mnb.var_)
print("Media por categoría:\n", u)
print("Desviación estándar por categoría:\n", std)

# Realizar una predicción
data_to_predict = np.array([[5,1,1,3,4,0]])
prediction = mnb.predict(data_to_predict)
print("Predicción para [5,1,1,3,4,0]:", prediction[0])


907       0
1580      0
1581      0
1586      0
1587      0
         ..
112598    6
112599    6
112600    6
112601    6
112602    6
Name: CrashType, Length: 2081, dtype: object
Distribución de los datos: [1580.  501.]
Total de datos: 2081
Porcentaje por categoría: [0.75925036 0.24074964]
Media por categoría:
 [[3.88417722 7.45696203 0.66265823 4.35189873 3.21772152 0.86265823]
 [3.9261477  4.1497006  0.62075848 6.69461078 2.27744511 0.79840319]]
Desviación estándar por categoría:
 [[1.95062542 3.52136997 0.47280262 2.3433645  1.87371962 0.34420782]
 [2.01603723 2.16198902 0.48519831 3.44163641 1.05765448 0.40119266]]
Predicción para [5,1,1,3,4,0]: 0


<ipython-input-10-896d4bda56b7>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sf_cortado.dropna(inplace=True)
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but GaussianNB was fitted with feature names
  warnings.warn(


In [ ]:
#Primer punto
num_datos = df_total.shape[0]
print(num_datos)
num_datos_sf = df_sf.shape[0]
print(num_datos_sf)

112660
2081


¿Cuál es el número de datos total de la base de datos?
La base de datos cuenta con 112,660 registros en total. Tras filtrar por San Francisco, quedaron 2,081 datos, asegurando un análisis específico para el modelo.

In [ ]:
#Segundo punto
print("Porcentaje por categoría:", ct / len(XD))


Porcentaje por categoría: [0.75925036 0.24074964]


¿Cuál es el porcentaje de los datos que hacen parte de cada una de las categorías de clasificación? El modelo Naive Bayes muestra que el 75.93% de los datos corresponden a la primera categoría de clasificación, mientras que el 24.07% pertenecen a la segunda. Esto indica un mayor número de accidentes en una de las dos categorías de autopista, lo que podría influir en la predicción del modelo.

In [ ]:
#Tercer punto
intervalos = df_sf_cortado.groupby('Highway')['Weekday'].agg(['mean', 'std'])

# Limites
limite_inferior = intervalos['mean'] - intervalos['std']
limite_superior = intervalos['mean'] + intervalos['std']

print(limite_inferior)
print(limite_superior)


Highway
0    1.932934
1    1.908095
dtype: float64
Highway
0    5.83542
1    5.94420
dtype: float64


Determinar los intervalos en los cual se encuentra la variación del día de la semana para cada una de las categorías de clasificación.  R/De acuerdo con los resultados del modelo, se observó que la variación del día de la semana para accidentes fuera de autopistas está en el intervalo [1.908095, 1.932934], mientras que en autopistas es [5.83542, 5.94420], indicando mayor dispersión en autopistas.

In [ ]:
#Cuarto punto
intervalos_violcat = df_sf_cortado.groupby('Highway')['ViolCat'].agg(['mean', 'std'])

limite_inferior_violcat = intervalos_violcat['mean'] - intervalos_violcat['std']
limite_superior_violcat = intervalos_violcat['mean'] + intervalos_violcat['std']

print(limite_inferior_violcat)
print(limite_superior_violcat)


Límites inferiores de ViolCat por categoría de Highway:
 Highway
0    3.934477
1    1.985551
dtype: float64
Límites superiores de ViolCat por categoría de Highway:
 Highway
0    10.979447
1     6.313851
dtype: float64


Determinar los intervalos en los cual se encuentra la variación de la ViolCat para cada una de las categorías de clasificación.  R/De acuerdo con los resultados del modelo, se observó que la variación de ViolCat para accidentes fuera de autopistas se encuentra en el intervalo [1.985551, 6.313851], mientras que en autopistas está en [3.934477, 10.979447], indicando un mayor rango de impacto en accidentes en autopistas.

In [ ]:
#Quinto punto
intervalos_crashtype = df_sf_cortado.groupby('Highway')['CrashType'].agg(['mean', 'std'])

limite_inferior_crashtype = intervalos_crashtype['mean'] - intervalos_crashtype['std']
limite_superior_crashtype = intervalos_crashtype['mean'] + intervalos_crashtype['std']

print(limite_inferior_crashtype)
print(limite_superior_crashtype)


Highway
0    1.343409
1    1.218734
dtype: object
Highway
0    5.092034
1    3.336157
dtype: object


Determinar el intervalo en los cuales se encuentra la variación de la Categoría de accidentes para cada una de las categorías de clasificación. R/De acuerdo con los resultados del modelo, la variación de la Categoría de Accidentes para accidentes fuera de autopistas se encuentra en el intervalo [1.218734, 1.343409], mientras que en autopistas está en [3.336157, 5.092034], mostrando una mayor dispersión en autopistas.

In [ ]:
#Sexto punto
Ex=(VN+VP)/(VN+FP+FN+VP)
print("la exactitud es:\n", Ex)

la exactitud es:
 0.7890437289764536


¿Cuál es la exactitud alcanzada por el modelo?.  De acuerdo con los resultados del modelo, se observó que la exactitud alcanzada es 78.90%, lo que indica un buen desempeño en la clasificación de accidentes.

In [ ]:
#Septimo punto
Esp=VN/(VN+FP)
print("la especificidad es:\n", Esp)

la especificidad es:
 0.8639240506329114


¿Cuál es la Especificidad alcanzada por el modelo?.  De acuerdo con los resultados del modelo, se observó que la especificidad alcanzada es 86.39%, lo que indica una alta capacidad para identificar correctamente los casos negativos.

In [ ]:
from sklearn.metrics import confusion_matrix

ydp = mnb.predict(XD)

#Octavo punto
cm=confusion_matrix(yd,ydp)
print("la matriz de confusion es:\n", cm)

VN=cm [0,0];FP=cm[0,1];FN=cm[1,0];VP=cm[1,1]

la matriz de confusion es:
 [[1365  215]
 [ 224  277]]


Indicar el intervalo en donde se ubica la cantidad de verdaderos negativo.
Indicar el intervalo en donde se ubica la cantidad de verdaderos positivos.  R/De acuerdo con la matriz de confusión, la cantidad de Verdaderos Negativos, se encuentra en el intervalo 1365, mientras que la cantidad de Verdaderos Positivos, se ubica en el intervalo 277. Esto refleja el desempeño del modelo en la clasificación de accidentes.


In [ ]:
#Noveno punto
TE=(FN+FP)/(VN+FP+FN+VP)
print("la tasa de error es:\n", TE)

la tasa de error es:
 0.21095627102354636


¿Cuál es la tasa de error alcanzada por el modelo?.  De acuerdo con los resultados del modelo, la tasa de error alcanzada es 21.10%, lo que indica el porcentaje de predicciones incorrectas en la clasificación de accidentes.

In [ ]:
#Decimo punto
Sen=VP/(VP+FN)
print("la sensibilidad es:\n", Sen)

la sensibilidad es:
 0.5528942115768463


¿Cuál es la Sensibilidad alcanzada por el modelo?.  De acuerdo con los resultados arrojados por el modelo, la Sensibilidad alcanzada por el modelo Naive Bayes es de 55.29%. Este valor indica la capacidad del modelo para identificar los accidentes que efectivamente ocurrieron en una autopista. Aunque la sensibilidad no es tan alta como otros indicadores, sigue siendo un resultado relevante dentro del contexto del análisis

In [ ]:
#Decimo primer punto
Pre=VP/(VP+FP)
print("la precision es:\n", Pre)

Pren=VN/(VN+FN)
print("la precision negativa es:\n", Pren)

la precision es:
 0.5630081300813008
la precision negativa es:
 0.8590308370044053


¿Cuál es la Precisión Alcanzada por el Modelo?.  La Precisión alcanzada por el modelo es de 56.30%, mientras que la Precisión Negativa es de 85.90%. Esto indica que el modelo tiene un mejor desempeño identificando correctamente los casos negativos, accidentes que no ocurrieron en autopistas, en comparación con los casos positivos accidentes en autopistas

In [ ]:
#Decimo segundo punto
XS=[[5,1,1,3,4,0]]
mnb.predict(XS)

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but GaussianNB was fitted with feature names
  warnings.warn(


array([0])

¿Indicar la categoría a la que pertenece una persona con las siguientes características [5,1,1,3,4,0]?.  De acuerdo con el resultado array([0]), una persona con las características [5, 1, 1, 3, 4, 0] pertenece a la categoría 0. Esto indica que, según el modelo, esta combinación de variables predice que la persona no tendra un accidente en la autopista